# Migrate to SDN4CoRE
This script translates the original NIDSDataSetCreation car network based on INET 4.5 to the SDN4CoRE simulation models.

## Migrate Apps

In [123]:
pubIds = []
subIds = []
createdMcastGroups = []
nextMcastGroup = 1
mcastBaseAddress = "224.123.45."

def isAppConfig(line):
    return ".app[" in line

def parseUniform(line):
    return line.split("=")[1].replace("uniform(", "").replace(")", "").strip().split(",")

def translateAppConfig(line):
    result = []
    line = line.replace(".app[", ".services[")
    if "packetNameFormat" in line:
        node = line.split(".")[1].strip()
        format = line.split("=")[1].replace("%M", node).replace("-%c", "").strip()
        result.append(line.split("=")[0].strip().replace("source.packetNameFormat", "display-name") + " = " + format)
        result.append(line.split("=")[0].strip().replace("source.packetNameFormat", "serviceName") + " = " + format)
    elif "typename" in line:
        if "UdpSourceApp" in line:
            result.append(line.replace("UdpSourceApp", "Publisher"))
        elif "UdpSinkApp" in line:
            result.append(line.replace("UdpSinkApp", "Subscriber"))
        else:
            result.append(line)
    elif "productionInterval" in line:
        if "uniform" in line:
            interval = parseUniform(line)
            parameter = line.split("=")[0].strip().replace("source.productionInterval", "intervalMin")
            if interval[0] == interval[1]:
                result.append(parameter + " = " + interval[0])
            else:
                result.append(parameter + " = " + min(interval[0], interval[1]))
                result.append(parameter.replace("Min", "Max") + " = " + max(interval[0], interval[1]))
        else:
            result.append(line.replace("source.productionInterval", "intervalMin"))
    elif "packetLength" in line:
        if "uniform" in line:
            length = parseUniform(line)
            parameter = line.split("=")[0].strip().replace("source.packetLength", "payloadMax")
            if length[0] == length[1]:
                result.append(parameter + " = " + length[0])
            else:
                result.append(parameter + " = " + max(length[0], length[1]))
                result.append(parameter.replace("Max", "Min") + " = " + min(length[0], length[1]))
        else:
            result.append(line.replace("source.packetLength", "payloadMax") + " - 24B")
    elif "destAddress" in line:
        if "224." in line:
            result.append(line.replace("io.destAddress", "mcastDestAddress"))
        else:
            global nextMcastGroup
            newAddress = mcastBaseAddress + str(nextMcastGroup)
            nextMcastGroup += 1
            result.append(line.split("=")[0].strip().replace("io.destAddress", "mcastDestAddress") + " = \"" + newAddress + "\"")
            print("Fix, this does not work with ** initializers! Maybe skip and make this a unicast publisher?")
    elif "destPort" in line:
        result.append(line.replace("io.destPort", "mcastDestPort"))
        result.append(line.replace("io.destPort", "serviceId"))
        serviceId = line.split("=")[1].strip()
        if serviceId not in pubIds:
            pubIds.append(serviceId)
        else:
            print("ServiceId already exists: " + serviceId + " in line: " + line)
    elif "localPort" in line:
        result.append(line.replace("io.localPort", "serviceId"))
        serviceId = line.split("=")[1].strip()
        if serviceId not in subIds:
            subIds.append(serviceId)
    else:
        result.append(line)
    return result

def translate(line):
    if line[0] == "#":
        return [line]
    comment = ""
    if "#" in line:
        comment = line.split("#")[1]
        line = line.split("#")[0]
    result = []
    if isAppConfig(line):
        newlines = translateAppConfig(line)
        for newline in newlines:
            result.append(newline)
    elif "numApps" in line:
        result.append(line.replace("numApps", "numServices"))
    else:
        result.append(line)
    if comment != "":
        for i in range(result.__len__()):
            result[i] = result[i].split("#")[0]
            if(i == 0):
                result[0] = result[0] + " #" + comment
    return result


In [124]:
def translateFile(infile, outfile):
    for line in infile:
        newlines = translate(line)
        for newline in newlines:
            if "\n" not in newline:
                newline = newline + "\n"
            outfile.write(newline)
    infile.close()
    outfile.close()

In [125]:
# read the ini file to modify
infile = open("omnetpp.ini", "r")
outfile = open("omnetpp_updated.ini", "w")
translateFile(infile, outfile)


In [126]:
# verify service ids
unused_subIds = [id for id in subIds if id not in pubIds]
for id in pubIds:
    if id not in subIds:
        print("ServiceId " + id + " is not used by any subscriber")
for id in unused_subIds:
    print("ServiceId " + id + " is not used by any publisher")


ServiceId 200 is not used by any publisher
ServiceId 300 is not used by any publisher
